How do we define death? ID is part of one census and not the next? IDEA: where in t2, DFstatus==dead OR agb==0

only 335 are labeled "dead" whereas 3,203 have an agb of 0 - which should we use?

In [22]:
import numpy as np
import scipy as sp
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
from sklearn.metrics import r2_score
import category_encoders as ce
import xgboost as xgb
import matplotlib.pyplot as mpl
from random import randint, choice

In [12]:
t1 = pd.read_csv("/Users/dylanvanbramer/indresearch/xu/deep_learning/bci_census/bci_1985.csv")
t2 = pd.read_csv("/Users/dylanvanbramer/indresearch/xu/deep_learning/bci_census/bci_1990.csv")


# load the first quadrat that we will be working with
quad1_t1 = t1.loc[t1['quadrat']<=50]
quad1_t2 = t2.loc[t2['quadrat']<=50]

# we only care about the tree IDs and DBHs of quad1_t2
expected_labels = quad1_t2[['treeID', 'agb', 'DFstatus' ]]
expected_labels = expected_labels.rename(columns={"treeID": "treeID2"})

#quad1_t1.head()
# simplify the data to have less features
quad1_t1 = quad1_t1[['treeID', 'sp', 'gx', 'gy', 'dbh']]
quad1_t1 = quad1_t1.rename(columns={"dbh": "dbh1", "treeID":"treeID1"})

/var/folders/zf/kbwgwbdd4gjfhdxc9cnw8qgm0000gn/T/ipykernel_24157/2191185153.py:1: DtypeWarning: Columns (14,16) have mixed types. Specify dtype option on import or set low_memory=False.
  t1 = pd.read_csv("/Users/dylanvanbramer/indresearch/xu/deep_learning/bci_census/bci_1985.csv")
/var/folders/zf/kbwgwbdd4gjfhdxc9cnw8qgm0000gn/T/ipykernel_24157/2191185153.py:2: DtypeWarning: Columns (14,16) have mixed types. Specify dtype option on import or set low_memory=False.
  t2 = pd.read_csv("/Users/dylanvanbramer/indresearch/xu/deep_learning/bci_census/bci_1990.csv")


# CLASSIFICATION PROBLEM - USING SAME DATA STRUCTURE AS NEIGHBORHOOD - trying to determine whether it dies or not in t2!!

Change all DEAD to 0, change all ALIVE to 1

In [14]:
expected_labels.loc[expected_labels['DFstatus'] == 'dead', 'DFstatus'] = 0
expected_labels.loc[expected_labels['DFstatus'] == 'missing', 'DFstatus'] = 0
expected_labels.loc[expected_labels['DFstatus'] == 'alive', 'DFstatus'] = 1

# also change any nonzero agb to 1 - more options later on for what we want to select as features
expected_labels.loc[expected_labels['agb'] > 0, 'agb'] = 1

In [15]:
encoder= ce.BinaryEncoder(cols=['sp'],return_df=True)
quad1_t1 = encoder.fit_transform(quad1_t1)
df_combined = pd.concat([quad1_t1, expected_labels], axis=1)

# Drop rows with any NaN values
df_combined_clean = df_combined.dropna()

# Separate the cleaned DataFrame and labels
df_clean = df_combined_clean[quad1_t1.columns]
labels_clean = df_combined_clean[expected_labels.columns]

quad1_t1 = df_clean.to_numpy()
expected_labels = labels_clean.to_numpy()

quad1_t1 = quad1_t1.astype(np.float32)
expected_labels = expected_labels.astype(np.float32)

In [16]:
# HOW MANY COLS TO ENCODE SPECIES?
a = 8

In [19]:
ids = quad1_t1[:,0]
# X WOULD BE a +1, y would be a+2
x_coordinates = quad1_t1[:, a+1]  
y_coordinates = quad1_t1[:, a+2]
coord_matrix = np.column_stack((x_coordinates, y_coordinates))
spatial_tree = sp.spatial.KDTree(coord_matrix)

nn_dist_matrix2 = np.zeros((len(coord_matrix),6))
nn_ind_matrix2 = np.zeros((len(coord_matrix),6))
nn_feats = np.column_stack((quad1_t1[:, 0:a+1], quad1_t1[:,a+3]))
feats_matrix = np.zeros((len(coord_matrix),(6*(a+2))))

for i, tree in enumerate(coord_matrix):
    dist2, ind2 = spatial_tree.query(tree, k=6)
    nn_ind_matrix2[i] = ids[ind2]
    nn_dist_matrix2[i]= dist2

    nn_row = nn_feats[i].reshape(1,a+2)
    inc = 0
    for j in nn_ind_matrix2[i][1:]:
        row_ind = np.where(quad1_t1[:,0] == j)

        real_row = (quad1_t1[row_ind])
        distance = dist2[1:][inc].reshape(1,1)
        dbh = real_row[:,a+3].reshape(1,1)
        nn_row = np.hstack((nn_row, distance, real_row[:,1:a+1],dbh))
        inc += 1
    
    feats_matrix[i] = nn_row
    
nn_ind_matrix2 = nn_ind_matrix2[:,1:]
nn_dist_matrix2 = nn_dist_matrix2[:,1:]

Randomization below:

In [20]:
rng = np.random.default_rng()
matrix_order = np.zeros((20,5))

shuffle_this = np.array([0, 1, 2, 3, 4])
for i in range(20):
  current_row = rng.permutation(shuffle_this)
  matrix_order[i] = current_row


nn_dist_matrix3 = np.zeros((len(coord_matrix),6))
nn_ind_matrix3 = np.zeros((len(coord_matrix),6))
nn_feats2 = np.column_stack((quad1_t1[:, 0:a+1], quad1_t1[:,a+3]))

# Start with feats_matrix above. Then create similar matrices for each of the above
feats_matrix_big = feats_matrix.copy()
# feats matrix small is miossing focal dbh

# of columns should be a*6

zer_indices = range(a+2,2*a+4)
one_indices = range (2*a+4,3*a+6)
two_indices = range (3*a+6,4*a+8)
three_indices = range (4*a+8,5*a+10)
four_indices = range (5*a+10,6*a+12)

# the long number passing into range is ust n=20 here, but this would make it less hard-coded in
for i in range(np.shape(matrix_order)[0]):
    # loop through each permutation.
    feats_matrix_small = feats_matrix.copy()
    perm_indices = [range(a+2)]
    current_perm = matrix_order[i]
    for x in current_perm:
        if x == 0:
          perm_indices.append(zer_indices)
        elif x == 1:
           perm_indices.append(one_indices)
        elif x == 2:
           perm_indices.append(two_indices)
        elif x == 3:
           perm_indices.append(three_indices)
        elif x == 4:
           perm_indices.append(four_indices)
    flattened = []
    for sublist in perm_indices:
       for item in sublist:
          flattened.append(item)
    feats_matrix_small = feats_matrix_small[:,flattened]
    feats_matrix_big = np.vstack((feats_matrix_big,feats_matrix_small))

expected_labels2 = expected_labels.copy()
for i in range(20):
    expected_labels2 = np.vstack((expected_labels2, expected_labels))

In [27]:
X_train, X_test, y_train, y_test = train_test_split(feats_matrix_big, expected_labels2, test_size=0.3)

feats = X_train[:,1:]
labels = y_train[:,2]

test_ids = X_test[:,0]
test_feats = X_test[:,1:]

In [37]:
xgb_tree = xgb.XGBClassifier()
xgb_tree.fit(feats,labels)
preds2 = xgb_tree.predict(test_feats)
predictions_matrix = np.column_stack((test_ids, preds2))
error2 = sklearn.metrics.mean_squared_error(y_test[:,2], preds2)
print (np.sqrt(error2))
error2a = r2_score (y_test[:,2], preds2)
print (error2a)

0.30453538675148867
0.1978992284021559


In [38]:
cv = KFold(n_splits=5)
cross_validate(xgb_tree, feats_matrix_big[:,1:], expected_labels2[:,2], scoring=('neg_root_mean_squared_error','r2'), cv=cv)

{'fit_time': array([10.27572203,  8.62734485,  8.465734  ,  8.25633192,  8.4396143 ]),
 'score_time': array([0.02098989, 0.01895499, 0.01958394, 0.02057409, 0.01789188]),
 'test_neg_root_mean_squared_error': array([-0.27738458, -0.3036631 , -0.29435542, -0.2941864 , -0.3016163 ]),
 'test_r2': array([0.32565988, 0.19027595, 0.23672422, 0.24005913, 0.21082811])}

XGBoost seems to work much better here! Things online suggest that this might be the case" without boosting, it will just predict majority.

In [36]:
rf2 = RandomForestClassifier(max_depth=8, n_estimators=100)
rf2.fit(feats,labels)

preds = rf2.predict(test_feats)
preds_matrix = np.column_stack((test_ids, preds))

error1 = sklearn.metrics.mean_squared_error(y_test[:,2], preds)
print (np.sqrt(error1))
error1a = r2_score(y_test[:,2], preds)
print (error1a)


cv_rf = RandomForestClassifier (max_depth=8)

scores = cross_validate(cv_rf, feats_matrix_big[:,1:], expected_labels2[:,2], scoring=('neg_root_mean_squared_error','r2'), cv=cv)
print(scores)

0.36527547
-0.15396979081506057
{'fit_time': array([7.50488877, 7.34737396, 7.10704303, 6.76504493, 6.78624821]), 'score_time': array([0.17878914, 0.19966912, 0.17845511, 0.18137503, 0.17780185]), 'test_neg_root_mean_squared_error': array([-0.36242768, -0.36201575, -0.36133716, -0.36202475, -0.36462584]), 'test_r2': array([-0.15121672, -0.1508214 , -0.1501716 , -0.15083   , -0.15333869])}
